In [0]:
%sql
select * from delta.`dbfs:/mnt/datalake/bronze/postgres/vendas/produtos`;
select * from delta.`dbfs:/mnt/datalake/bronze/postgres/vendas/itens_pedido`;
select * from delta.`dbfs:/mnt/datalake/bronze/postgres/vendas/clientes`;
select * from delta.`dbfs:/mnt/datalake/bronze/postgres/vendas/pedido`;


In [0]:
dbutils.fs.mkdirs("dbfs:/mnt/datalake/silver/vendas/clientes")
dbutils.fs.mkdirs("dbfs:/mnt/datalake/silver/vendas/produtos")
dbutils.fs.mkdirs("dbfs:/mnt/datalake/silver/vendas/pedidos")
dbutils.fs.mkdirs("dbfs:/mnt/datalake/silver/vendas/itens_pedido")


In [0]:
%sql
--Tabela Dimensão clientes

CREATE OR REPLACE TABLE silver_clientes
USING DELTA
LOCATION 'dbfs:/mnt/datalake/silver/vendas/clientes'
AS
SELECT DISTINCT 
    id AS cliente_id, 
    nome, 
    email, 
    cidade
FROM delta.`dbfs:/mnt/datalake/bronze/postgres/vendas/clientes`;

--Tabela Dimensão produtos 

CREATE OR REPLACE TABLE silver_produtos
USING DELTA
LOCATION 'dbfs:/mnt/datalake/silver/vendas/produtos'
AS
SELECT DISTINCT 
    id AS produto_id, 
    nome, 
    categoria, 
    cast(preco as double) as Preco
FROM delta.`dbfs:/mnt/datalake/bronze/postgres/vendas/produtos`;

--Tabela Dimensão pedidos

-- Preciso retirar Clientes_id para dimensão silver

CREATE OR REPLACE TABLE silver_pedidos
USING DELTA
LOCATION 'dbfs:/mnt/datalake/silver/vendas/pedidos'
AS
SELECT DISTINCT 
    id AS Pedido_id, 
    CAST(data as date) as Data_pedido, 
    CAST(valor_total as double) AS Valor_total
FROM delta.`dbfs:/mnt/datalake/bronze/postgres/vendas/pedido`;

-- Tabela Fato - Itens Pedido

-- # Para criar a tabela fato vou pegar a coluna cliente fazendo um join da tabela pedido que possui uma fk pedido_id na tabela itens_pedido, porém vou usar a tabela do lake bronze , assim consigo criar uma tabela delta trazer a coluna clientes_id do pedido para tabela fato itens pedido 


CREATE OR REPLACE TABLE silver_itens_pedido
USING DELTA 
LOCATION 'dbfs:/mnt/datalake/silver/vendas/itens_pedido_2'
AS
SELECT
    i.id as itens_id,
    p.cliente_id,
    i.produto_id,
    i.pedido_id,
    i.quantidade,
    i.preco_unitario
FROM delta.`dbfs:/mnt/datalake/silver/vendas/itens_pedido` i
JOIN delta.`dbfs:/mnt/datalake/bronze/postgres/vendas/pedido` p
ON i.pedido_id = p.id

    


In [0]:
# %fs ls dbfs:/mnt/datalake/bronze/postgres/vendas/
# %fs ls dbfs:/mnt/datalake/silver/vendas/

In [0]:
%sql

SELECT * FROM delta.`dbfs:/mnt/datalake/silver/vendas/produtos/`;
SELECT * FROM delta.`dbfs:/mnt/datalake/silver/vendas/clientes/`;
SELECT * FROM delta.`dbfs:/mnt/datalake/silver/vendas/pedidos/`;
SELECT * FROM delta.`dbfs:/mnt/datalake/silver/vendas/itens_pedido_2/`;
